In [3]:
import pandas as pd           # data mnipulation
import numpy as np            # number manipulation/crunching
import matplotlib.pyplot as plt  # plotting
from sklearn.metrics import  accuracy_score, classification_report 
import openpyxl
import sklearn
# Train Test split
from sklearn.model_selection import train_test_split
# Random forest classifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from anchor import utils
from anchor import anchor_tabular
from sklearn.model_selection import LeaveOneOut
import lime
import lime.lime_tabular
from collections import Counter
import shap
import math

fetal = pd.read_csv("fetal.csv")
fetal["Outcome"].value_counts()

Y = fetal['Outcome']
target_names=["0", "D1"]
X = fetal[['accelerations',	'fetal_movement',	'uterine_contractions',	'light_decelerations',	'severe_decelerations',	'prolongued_decelerations',
              'abnormal_short_term_variability',	'mean_value_of_short_term_variability',	'percentage_of_time_with_abnormal_long_term_variability',
              'mean_value_of_long_term_variability',	'histogram_width',	'histogram_min',	'histogram_max',	'histogram_number_of_peaks',
              'histogram_number_of_zeroes',	'histogram_mode',	'histogram_mean',	'histogram_median',	'histogram_variance',	'histogram_tendency'	]]
maximums = np.amax(np.array(X), axis=0)
X_featurenames = X.columns

# Split the data into train and test data:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, shuffle = True)
counter = Counter(Y)
model = sklearn.svm.SVC(kernel='rbf', probability=True)
model.fit(X_train, Y_train)
pred = model.predict(X_test)
accuracy = accuracy_score(Y_test, pred)
print("Accuracy:", accuracy)
print(classification_report(Y_test, pred, target_names=["0", "1"]))

# model = KNeighborsClassifier(n_neighbors=11)
# model.fit(X_train, Y_train)
# predicted_knn = model.predict(X_test)
# accuracy_knn = accuracy_score(Y_test, predicted_knn)
# print("Precisión Knn ", accuracy_knn)

shap.initjs()

dataframe_columns_Shap = []
test_index = "test_index"
dataframe_columns_Shap.append(test_index)
for feature in X_featurenames:
    feature_shap_value = feature + "_shap_value"
    feature_sign = feature + "_sign"
    feature_rank = feature + "_rank"
    dataframe_columns_Shap.append(feature_shap_value)
    dataframe_columns_Shap.append(feature_sign)
    dataframe_columns_Shap.append(feature_rank)
expected_value_0 = "expected_value_0"
expected_value_1 = "expected_value_1"
f_0 = "f_0"
f_1 = "f_1"
probability_no_fetal_problems = "probability_no_fetal_problems"
probability_fetal_problems = "probability_fetal_problems"
accuracy_classifier = "accuracy_classifier"
true_class = "true_class"
dataframe_columns_Shap.append(expected_value_0)
dataframe_columns_Shap.append(expected_value_1)
dataframe_columns_Shap.append(f_0)
dataframe_columns_Shap.append(f_1)
dataframe_columns_Shap.append(probability_no_fetal_problems)
dataframe_columns_Shap.append(probability_fetal_problems)
dataframe_columns_Shap.append(accuracy_classifier)
dataframe_columns_Shap.append(true_class)
df_Shap = pd.DataFrame(columns = dataframe_columns_Shap)


cv = LeaveOneOut()
number_iterations = 1

for train_ix, test_ix in cv.split(X):
    print(test_ix[0])
    for count in range(number_iterations):
        row_Shap = []
        row_Shap.append(test_ix[0])
        model = sklearn.svm.SVC(kernel='rbf', probability=True)
        X_train, X_test = X.to_numpy()[train_ix, :], X.to_numpy()[test_ix, :]
        Y_train, Y_test = Y[train_ix], Y[test_ix]
        model.fit(X_train, Y_train)
        # a summarized version of the dataset using shap.kmeans. This has been done to reduce computation time
        explainer = shap.KernelExplainer(model.predict_proba, shap.kmeans(X_train, 10))
        #explainer = shap.KernelExplainer(model.predict_proba, X_train)
        shap_values = explainer.shap_values(X_test)
        #print(shap_values)
        shap_values = np.delete(shap_values, obj = 1, axis = 0).flatten()
        #print(shap_values)
        predict_proba = model.predict_proba(X_test).flatten()
        #print(predict_proba)
        ranks = []
        for i in range(len(X_featurenames)):
            count_lower = 0
            for j in range(len(X_featurenames)):
                if abs(shap_values[j]) < abs(shap_values[i]):
                    count_lower = count_lower + 1
            position = len(X_featurenames) - count_lower
            row_Shap.append(shap_values[i])
            if shap_values[i] > 0 :
                sign = "positive"
            else :
                sign = "negative"
            row_Shap.append(sign)
            row_Shap.append(position)
            ranks.append(position)
        #print(ranks)
        #print(X_featurenames)
        #print(explainer.expected_value)
        row_Shap.append(explainer.expected_value[0])
        row_Shap.append(explainer.expected_value[1])
        sum_shap_values = np.array(shap_values).sum(0) 
        f_0 = explainer.expected_value[0] + sum_shap_values
        f_1 = explainer.expected_value[1] - sum_shap_values
        #print(f_0)
        #print(f_1)
        row_Shap.append(f_0)
        row_Shap.append(f_1)
        probability_no_fetal_problems = predict_proba[0]
        probability_fetal_problems = predict_proba[1]
        row_Shap.append(probability_no_fetal_problems)
        row_Shap.append(probability_fetal_problems)
        pred = model.predict(X_test)
        accuracy_classifier = accuracy_score(Y_test, pred)
        true_class = target_names[Y_test[test_ix[0]]]
        row_Shap.append(accuracy_classifier)
        row_Shap.append(true_class)
        #print(row_Shap)
        df_Shap.loc[test_ix[0] * number_iterations + count] = row_Shap
        
df_Shap.to_excel("outputFetalKernelShapLOOCV.xlsx")

Accuracy: 0.8793103448275862
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       505
           1       0.79      0.58      0.67       133

    accuracy                           0.88       638
   macro avg       0.84      0.77      0.80       638
weighted avg       0.87      0.88      0.87       638



0


  0%|          | 0/1 [00:00<?, ?it/s]

1


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

2124


  0%|          | 0/1 [00:00<?, ?it/s]

2125


  0%|          | 0/1 [00:00<?, ?it/s]